# Proyecto 1
## Detección de ataques

In [2]:
import os.path

import pandas as pd
from pandas_profiling import ProfileReport

import utils

In [ ]:
generate_reports: bool = False

In [3]:
part_1 = pd.read_csv("dataset/dataset-part1.csv")
part_2 = pd.read_csv("dataset/dataset-part2.csv")

In [4]:
df = pd.concat([part_1, part_2], ignore_index=True)

## Análisis Exploratorio

In [5]:
df.columns.values

array(['BIFLOW_DIRECTION', 'DIRECTION', 'DST_TO_SRC_SECOND_BYTES',
       'FIREWALL_EVENT', 'FIRST_SWITCHED', 'FLOW_ACTIVE_TIMEOUT',
       'FLOW_DURATION_MICROSECONDS', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_END_SEC', 'FLOW_ID',
       'FLOW_INACTIVE_TIMEOUT', 'FLOW_START_MILLISECONDS',
       'FLOW_START_SEC', 'FRAME_LENGTH', 'IN_BYTES', 'IN_PKTS',
       'IPV4_DST_ADDR', 'IPV4_SRC_ADDR', 'L4_DST_PORT', 'L4_SRC_PORT',
       'LAST_SWITCHED', 'MAX_IP_PKT_LEN', 'MIN_IP_PKT_LEN',
       'OOORDER_IN_PKTS', 'OOORDER_OUT_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'PROTOCOL', 'PROTOCOL_MAP', 'RETRANSMITTED_IN_BYTES',
       'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_BYTES',
       'RETRANSMITTED_OUT_PKTS', 'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS',
       'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN',
       'TCP_WIN_MIN_OUT', 'TCP_WIN_MSS_IN', 'TCP_WIN_MSS_OUT',
       'TCP_WIN_SCALE_IN', 'TCP_WIN_SCALE_OUT', 'SRC_TOS', 'DST_TOS',
       'L7_PROTO_NAME', 'SAM

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12207873 entries, 0 to 12207872
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  

In [7]:
df.head()

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow


In [8]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BIFLOW_DIRECTION,12207873.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
DIRECTION,12207873.0,2.158525e-03,4.640976e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
FIREWALL_EVENT,12207873.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
FIRST_SWITCHED,12207873.0,1.617770e+09,7.090651e+05,1.616660e+09,1.616673e+09,1.618222e+09,1.618232e+09,1.618262e+09
FLOW_ACTIVE_TIMEOUT,12207873.0,1.200000e+02,0.000000e+00,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02
FLOW_DURATION_MICROSECONDS,12207873.0,2.162737e+07,3.997485e+07,0.000000e+00,4.610000e+02,2.998000e+04,1.373936e+07,1.199999e+08
FLOW_DURATION_MILLISECONDS,12207873.0,2.162695e+04,3.997481e+04,0.000000e+00,0.000000e+00,2.900000e+01,1.373900e+04,1.199990e+05
FLOW_END_MILLISECONDS,12207873.0,1.617770e+12,7.090782e+08,1.616660e+12,1.616673e+12,1.618222e+12,1.618232e+12,1.618263e+12
FLOW_END_SEC,12207873.0,1.617770e+09,7.090782e+05,1.616660e+09,1.616673e+09,1.618222e+09,1.618232e+09,1.618263e+09
FLOW_ID,12207873.0,5.890479e+07,3.531791e+07,2.293398e+06,5.345536e+06,7.978476e+07,8.283673e+07,8.588870e+07


In [9]:
df.describe(include=['object'])

,DST_TO_SRC_SECOND_BYTES,IPV4_DST_ADDR,IPV4_SRC_ADDR,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,L7_PROTO_NAME,LABEL
count,12207873,12207873,12207873,12207873,12207873,12207873,12207873
unique,569243,440887,88463,5,3160289,457,4
top,40,10.114.224.65,10.114.241.166,tcp,44,Unknown,Normal flow
freq,3053652,3604686,5671376,8950058,2534718,4090802,6570058


In [10]:
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns].head(10)

,DST_TO_SRC_SECOND_BYTES,IPV4_DST_ADDR,IPV4_SRC_ADDR,PROTOCOL_MAP,SRC_TO_DST_SECOND_BYTES,L7_PROTO_NAME,LABEL
0,40,10.114.225.212,162.142.125.173,tcp,44,Unknown,Normal flow
1,",",10.114.225.215,45.79.106.170,icmp,68,ICMP,Normal flow
2,104,10.114.241.165,10.114.224.65,tcp,189,TLS,Normal flow
3,",",10.114.226.23,202.179.91.28,tcp,52,Unknown,Normal flow
4,40,10.114.224.65,10.114.241.165,tcp,189,TLS,Normal flow
5,40,10.114.224.65,10.114.241.165,tcp,189,TLS,Normal flow
6,5043,172.217.16.133,10.114.225.206,tcp,3699,TLS.Google,Normal flow
7,1009,52.114.77.99,10.114.225.205,tcp,4861,TLS.Teams,Normal flow
8,",",10.114.226.5,14.236.174.82,tcp,52,SMBv23,Normal flow
9,120,52.113.194.132,10.114.225.204,tcp,5707,TLS.Skype,Normal flow


In [11]:
df["LABEL"].unique()

array(['Normal flow', 'SYN Scan - aggressive',
       'Denial of Service R-U-Dead-Yet', 'Denial of Service Slowloris'],
      dtype=object)

In [12]:
# Check for nan columns
df.columns[df.isna().any()].tolist()
# No nan

[]

In [13]:
if not os.path.isfile("cleanEDA.html") and generate_reports:
    profile = ProfileReport(df, title="Exploratory Data Analysis - Clean DF", minimal=True)
    profile.to_file("cleanEDA.html")

## Preprocessing

### Remove unnecesary variables

#### Drop constants

In [14]:
# CONSTANTS 
constants = [
    'BIFLOW_DIRECTION',
    'FIREWALL_EVENT',
    'FLOW_ACTIVE_TIMEOUT',
    'FLOW_INACTIVE_TIMEOUT',
    'FRAME_LENGTH',
    'MAX_IP_PKT_LEN',
    'MIN_IP_PKT_LEN',
    'SAMPLING_INTERVAL',
    'DIRECTION',
    'OOORDER_IN_PKTS',
    'OOORDER_OUT_PKTS',
]

# DIRECTION, OOORDER_IN_PKTS and OOORDER_OUT_PKTS has 99%> zero values, so it will be considered a constant.

In [15]:
# Drop constants from dataframe
df = df.drop(constants, axis=1)
len(df.columns.values)

39

#### Drop unique values

In [16]:
# UNIQUE
unique_vars = ['FLOW_ID', 'TOTAL_FLOWS_EXP']

In [17]:
df = df.drop(unique_vars, axis=1)
len(df.columns.values)

37

#### DROP IPS

In [18]:
df = df.drop(["IPV4_DST_ADDR", "IPV4_SRC_ADDR"], axis = 1)

#### DROP PROTOCOL COLUMN

In [19]:
df = df.drop(["PROTOCOL"], axis = 1)

### Numerical preprocessing

#### Fix DST_TO_SRC_SECOND_BYTES column

In [20]:
# df["DST_TO_SRC_SECOND_BYTES"].apply(np.isreal)
DST_TO_SRC_SECOND_BYTES_INT_VALUES = list(filter(utils.is_int, df["DST_TO_SRC_SECOND_BYTES"]))
DST_TO_SRC_SECOND_BYTES_NON_INT_VALUES = list(filter(lambda x: not utils.is_int(x), df["DST_TO_SRC_SECOND_BYTES"]))

In [21]:
print("Correct int values: ", len(DST_TO_SRC_SECOND_BYTES_INT_VALUES))
print("Incorrect int values: ", len(DST_TO_SRC_SECOND_BYTES_NON_INT_VALUES))
print("Total values count: ", len(df["DST_TO_SRC_SECOND_BYTES"]))

Correct int values:  5994780
Incorrect int values:  6213093
Total values count:  12207873


In [22]:
df["DST_TO_SRC_SECOND_BYTES"] = df["DST_TO_SRC_SECOND_BYTES"].apply(lambda x: int(x) if utils.is_int(x) else utils.normalize_dst_to_src_column(x))
df["DST_TO_SRC_SECOND_BYTES"].head()

0     40
1      0
2    104
3      0
4     40
Name: DST_TO_SRC_SECOND_BYTES, dtype: int64

#### Fix SRC_TO_DST_SECOND_BYTES column

In [23]:
df["SRC_TO_DST_SECOND_BYTES"] = df["SRC_TO_DST_SECOND_BYTES"].apply(lambda x: int(x) if utils.is_int(x) else utils.normalize_dst_to_src_column(x))
df["SRC_TO_DST_SECOND_BYTES"].head()

0     44
1     68
2    189
3     52
4    189
Name: SRC_TO_DST_SECOND_BYTES, dtype: int64

### Second profile report

In [24]:
if not os.path.isfile("profile-numerical.html") and generate_reports:
    profile = ProfileReport(df, title="Profile after numerical preprocessing", minimal=True)
    profile.to_file("profile-numerical.html")

### Categorical variables preprocessing

#### Create PROTOCOL_MAP dummies

In [25]:
# Create the dummies
df = pd.get_dummies(df, columns = ["PROTOCOL_MAP"])
df.head(3)

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,...,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,LABEL,PROTOCOL_MAP_gre,PROTOCOL_MAP_icmp,PROTOCOL_MAP_ipv6-icmp,PROTOCOL_MAP_tcp,PROTOCOL_MAP_udp
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,...,0,0,0,Unknown,Normal flow,0,0,0,1,0
1,0,1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,...,0,40,0,ICMP,Normal flow,0,1,0,0,0
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,...,0,0,0,TLS,Normal flow,0,0,0,1,0


#### L7_PROTO_NAME dummies

In [26]:
df["L7_PROTO_NAME"].unique()

array(['Unknown', 'ICMP', 'TLS', 'TLS.Google', 'TLS.Teams', 'SMBv23',
       'TLS.Skype', 'DNS.Teams', 'Google', 'DNS.Google', 'QUIC.Google',
       'DNS.OpenDNS', 'DNS', 'DNS.Microsoft365', 'DNS.Microsoft',
       'HTTP_Proxy', 'NFS', 'TLS.Dropbox', 'DNS.Cloudflare', 'SSH', 'RDP',
       'TLS.Cloudflare', 'Telnet', 'BitTorrent', 'BGP', 'HTTP', 'NTP',
       'DNS.Yahoo', 'TLS.Microsoft365', 'MsSQL-TDS', 'DNS.Dropbox',
       'TeamViewer', 'Memcached', 'TLS.Amazon', 'TLS.Telegram', 's7comm',
       'DNS.WhatsApp', 'TLS.WhatsApp', 'MDNS', 'TLS.Microsoft',
       'HTTP.Google', 'SIP', 'DNS.GoogleServices', 'TLS.GoogleServices',
       'NTP.Cloudflare', 'HTTP.TeamViewer', 'LDAP', 'TLS.WhatsAppFiles',
       'DNS.Amazon', 'SNMP', 'COAP.DTLS', 'DNS.Facebook', 'TLS.Yahoo',
       'TLS.TeamViewer', 'QUIC.Facebook', 'DNS.MS_OneDrive', 'DNS.Skype',
       'TLS.Tor', 'Amazon', 'IMAPS', 'TLS.MS_OneDrive', 'CiscoSkinny',
       'WhatsAppFiles', 'DNS.UbuntuONE', 'HTTP.UbuntuONE',
       'DNS.GoogleD

In [27]:
df["L7_PROTO_NAME"] = df["L7_PROTO_NAME"].map(lambda protocol: utils.lematize_protocol(protocol).upper())

In [28]:
print(len(df["L7_PROTO_NAME"].unique()))

df["L7_PROTO_NAME"].unique()

132


array(['UNKNOWN', 'ICMP', 'TLS', 'SMBV23', 'DNS', 'GOOGLE', 'QUIC',
       'HTTP_PROXY', 'NFS', 'SSH', 'RDP', 'TELNET', 'BITTORRENT', 'BGP',
       'HTTP', 'NTP', 'MSSQL-TDS', 'TEAMVIEWER', 'MEMCACHED', 'S7COMM',
       'MDNS', 'SIP', 'LDAP', 'SNMP', 'COAP', 'AMAZON', 'IMAPS',
       'CISCOSKINNY', 'WHATSAPPFILES', 'POPS', 'SSDP', 'RTSP', 'POP3',
       'VNC', 'NETBIOS', 'FTP_CONTROL', 'H323', 'SMTP', 'MYSQL',
       'DCE_RPC', 'SMTPS', 'IMAP', 'REMOTESCAN', 'IAX', 'IEC60870',
       'MQTT', 'APPLE', 'RTP', 'MONGODB', 'FTP_DATA', 'SOMEIP',
       'POSTGRESQL', 'MODBUS', 'OPENVPN', 'STEAM', 'CISCOVPN',
       'PLAYSTATION', 'SKYPE', 'NESTLOGSINK', 'DNP3', 'STUN', 'WSD',
       'XBOX', 'TARGUSDATASPEED', 'AFP', 'LOTUSNOTES', 'KERBEROS',
       'RADIUS', 'SYSLOG', 'WHOIS-DAS', 'TFTP', 'IPSEC', 'IRC', 'OSPF',
       'XDMCP', 'MINING', 'VIBER', 'REDIS', 'ZABBIX', 'AJP', 'SOCKS',
       'LLMNR', 'TOR', 'ORACLE', 'CHECKMK', 'CITRIX', 'SAP', 'STARCRAFT',
       'GIT', 'DOH_DOT', 'RTMP', 'RSYNC

In [29]:
df = pd.get_dummies(df, columns = ["L7_PROTO_NAME"])
df.head(3)

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,...,L7_PROTO_NAME_WEBEX,L7_PROTO_NAME_WHATSAPP,L7_PROTO_NAME_WHATSAPPFILES,L7_PROTO_NAME_WHOIS-DAS,L7_PROTO_NAME_WIREGUARD,L7_PROTO_NAME_WSD,L7_PROTO_NAME_XBOX,L7_PROTO_NAME_XDMCP,L7_PROTO_NAME_ZABBIX,L7_PROTO_NAME_ZOOM
0,40,1616660040,339,0,1616660040010,1616660040,1616660040010,1616660040,44,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1616660040,0,0,1616660040068,1616660040,1616660040068,1616660040,68,1,...,0,0,0,0,0,0,0,0,0,0
2,104,1616660040,44725,44,1616660040114,1616660040,1616660040070,1616660040,189,2,...,0,0,0,0,0,0,0,0,0,0


#### Treat LABEL column

In [30]:
df["LABEL"].unique()

array(['Normal flow', 'SYN Scan - aggressive',
       'Denial of Service R-U-Dead-Yet', 'Denial of Service Slowloris'],
      dtype=object)

In [31]:
labels_dic = {
    "Normal flow": 0,
    "SYN Scan - aggressive": 1,
    "Denial of Service R-U-Dead-Yet": 2,
    "Denial of Service Slowloris": 3
}
df["LABEL"] = df["LABEL"].apply(lambda x: labels_dic[x])
df["LABEL"].unique()

array([0, 1, 2, 3])

### Normalize column names

In [34]:
# Normalize columns
df.columns = [column.replace("-", "_") for column in df.columns]
df.columns

Index(['DST_TO_SRC_SECOND_BYTES', 'FIRST_SWITCHED',
       'FLOW_DURATION_MICROSECONDS', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_END_SEC', 'FLOW_START_MILLISECONDS',
       'FLOW_START_SEC', 'IN_BYTES', 'IN_PKTS',
       ...
       'L7_PROTO_NAME_WEBEX', 'L7_PROTO_NAME_WHATSAPP',
       'L7_PROTO_NAME_WHATSAPPFILES', 'L7_PROTO_NAME_WHOIS_DAS',
       'L7_PROTO_NAME_WIREGUARD', 'L7_PROTO_NAME_WSD', 'L7_PROTO_NAME_XBOX',
       'L7_PROTO_NAME_XDMCP', 'L7_PROTO_NAME_ZABBIX', 'L7_PROTO_NAME_ZOOM'],
      dtype='object', length=169)

### Third profile report

In [ ]:
df.shape()

In [35]:
if not os.path.isfile("profile-categorical.html") and generate_reports:
    profile = ProfileReport(df, title="Exploratory Data Analysis 2")
    profile.to_file("profile-categorical.html")

### Dataset balancing

In [41]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(sampling_strategy='minority')

In [ ]:
normal_flow = df.loc[df["LABEL"] == 0]
syn_attacks = df.loc[df["LABEL"] == 1]
r_u_dead_attacks = df.loc[df["LABEL"] == 2]
dos_attacks = df.loc[df["LABEL"] == 3]

In [ ]:
print(len(normal_flow))
print(len(syn_attacks))
print(len(r_u_dead_attacks))
print(len(dos_attacks))

#### Oversampling DDoS attack

In [42]:
attacks_oversampling = pd.concat([syn_attacks, dos_attacks])
y_oversampling = attacks_oversampling["LABEL"]
x_oversampling = attacks_oversampling.drop(["LABEL"], axis= 1)

In [43]:
y_oversampling

3060        1
3071        1
3073        1
3074        1
3076        1
           ..
11513523    3
11513526    3
11513568    3
11513569    3
11513579    3
Name: LABEL, Length: 3360868, dtype: int64

In [44]:
x_oversampling

,DST_TO_SRC_SECOND_BYTES,FIRST_SWITCHED,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_START_MILLISECONDS,FLOW_START_SEC,IN_BYTES,IN_PKTS,...,L7_PROTO_NAME_WEBEX,L7_PROTO_NAME_WHATSAPP,L7_PROTO_NAME_WHATSAPPFILES,L7_PROTO_NAME_WHOIS-DAS,L7_PROTO_NAME_WIREGUARD,L7_PROTO_NAME_WSD,L7_PROTO_NAME_XBOX,L7_PROTO_NAME_XDMCP,L7_PROTO_NAME_ZABBIX,L7_PROTO_NAME_ZOOM
3060,40,1616660094,721,0,1616660094822,1616660094,1616660094821,1616660094,44,1,...,0,0,0,0,0,0,0,0,0,0
3071,40,1616660094,622,0,1616660094921,1616660094,1616660094921,1616660094,44,1,...,0,0,0,0,0,0,0,0,0,0
3073,40,1616660094,214,0,1616660094925,1616660094,1616660094925,1616660094,44,1,...,0,0,0,0,0,0,0,0,0,0
3074,40,1616660094,635,0,1616660094926,1616660094,1616660094925,1616660094,44,1,...,0,0,0,0,0,0,0,0,0,0
3076,40,1616660094,655,0,1616660094930,1616660094,1616660094929,1616660094,44,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11513523,216,1618247699,9224370,9224,1618247708743,1618247708,1618247699519,1618247699,593,6,...,0,0,0,0,0,0,0,0,0,0
11513526,320,1618247677,30918092,30918,1618247708743,1618247708,1618247677825,1618247677,764,8,...,0,0,0,0,0,0,0,0,0,0
11513568,268,1618247685,23500256,23500,1618247708745,1618247708,1618247685244,1618247685,688,7,...,0,0,0,0,0,0,0,0,0,0
11513569,320,1618247679,29680842,29680,1618247708745,1618247708,1618247679064,1618247679,768,8,...,0,0,0,0,0,0,0,0,0,0


In [45]:
X_oversampling_res, y_oversampling_res = ros.fit_resample(x_oversampling, y_oversampling)

In [ ]:
print('Resampled dataset shape %s' % Counter(y_oversampling_res))

Resampled dataset shape Counter({1: 2496814, 3: 2496814})


In [ ]:
oversampling_result = X_oversampling_res.copy()
oversampling_result["LABEL"] = y_oversampling_res.copy()

In [ ]:
sampled_df = pd.concat([normal_flow, r_u_dead_attacks, oversampling_result])

#### Undersampling benign

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
normal_flow = df.loc[df["LABEL"] == 0]
syn_attacks = df.loc[df["LABEL"] == 1]
r_u_dead_attacks = df.loc[df["LABEL"] == 2]
dos_attacks = df.loc[df["LABEL"] == 3]

In [ ]:
benign_us = pd.concat([syn_attacks, normal_flow])

In [ ]:
y = benign_us["LABEL"]
X = benign_us.drop(["LABEL"], axis=1)

In [ ]:
rus = RandomUnderSampler(random_state=0)

In [ ]:
X_undersampled, y_undersampled = rus.fit_resample(X, y)

print('Resampled dataset shape %s' % Counter(y_undersampled))

In [ ]:
X_undersampled["LABEL"] = y_undersampled
undersample_res = X_undersampled

In [ ]:
sampled_df = pd.concat([dos_attacks, r_u_dead_attacks, undersample_res])

### Create clean dataset

In [ ]:
sampled_df.to_csv("dataset/clean_df.csv")